Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'
Reference_Data_database = 'aaa'


pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:56:11.067438
svmem(total=16756752384, available=4337516544, percent=74.1, used=12419235840, free=4337516544)


# 1. Get Broker and Commissions features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

broker_query = f"""SELECT pol.Policy_Number AS ContractNumber
	  ,pol.Servicing_Broker_Id AS ServicingBroker
      ,pol.Initial_Broker_Id   AS InitialBroker
      ,pb.Broker_Category_En AS BrokerCategory
	  ,AVG(cr.Initial_Rate) AS InitialRate
	  ,AVG(cr.renewal_Rate) AS RenewalRate
FROM Policy pol
INNER JOIN Product prd ON prd.Product_Id = pol.Product_Id
INNER JOIN Policy_Broker pb ON pb.Statement_Broker_Id = pol.Servicing_Broker_Id
INNER JOIN Commission_Rate cr ON cr.Policy_Broker_Id = pb.Policy_Broker_Type_Id --AND cr.Policy_Broker_Id = pb.Policy_Broker_Id
WHERE prd.Product_Id = pol.Product_Id
AND   prd.Product_Code IN ('FKP','GNPP_','GNRP')
AND SUBSTRING(prd.Product_Category_Code,1,3) = SUBSTRING(cr.Commissions_Product_Group,1,3)
GROUP BY pol.Policy_Number,pol.Servicing_Broker_Id,pol.Initial_Broker_Id,pb.Broker_Category_En;"""

df_broker = pd.read_sql(broker_query, conn)

# Types and row count
# print stats.
print(len(df_broker.index))
now = datetime.now()
print(now)
print('==========================')


print(df_broker.dtypes)

# No displays to protect PII data
#df_broker.head(5)

69683
2024-08-14 17:56:12.777567
ContractNumber      object
ServicingBroker      int64
InitialBroker        int64
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object


## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_broker.dtypes

ContractNumber      object
ServicingBroker      int64
InitialBroker        int64
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object

In [5]:
df_broker.isna().any()

ContractNumber     False
ServicingBroker    False
InitialBroker      False
BrokerCategory     False
InitialRate        False
RenewalRate        False
dtype: bool

## All features well distributed so write to csv.

In [6]:
# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Broker_and_Commission_data.csv'
df_broker.to_csv(filename, encoding='utf-8', index=False)        

# No displays to protect PII data
#df_broker.head(10)

In [7]:
%who_ls DataFrame 

['df_broker']

In [8]:
del df_broker





In [9]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:56:13.186487
svmem(total=16756752384, available=4329365504, percent=74.2, used=12427386880, free=4329365504)
